In [1]:
library(BSgenome.Hsapiens.UCSC.hg38)
library(data.table)
for (i in names(Hsapiens)[1:24]){ # load all chromosome map files
        cat(paste0('Loading...',i,'\n'))
        load(paste0('./sim-develop/data/root_maps/',i,'.rda'))
}
load('./sim-develop/data/exann.rda')

Loading required package: BSgenome
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors
Loading require

Loading...chr1


Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file './sim-develop/data/root_maps/chr1.rda', probable reason 'No such file or directory'”

ERROR: Error in readChar(con, 5L, useBytes = TRUE): cannot open the connection


In [2]:
head(exann)

chrom,start,end,gene_id,istsg,geneSym
chr1,11874,12227,100287102,0,DDX11L1
chr1,12595,12721,100287102,0,DDX11L1
chr1,13221,16765,100287102,0,DDX11L1
chr1,16854,17055,653635,0,WASH7P
chr1,17233,18061,653635,0,WASH7P
chr1,18268,18379,653635,0,WASH7P


In [36]:
counts <-  array(0,dim=c(24,12)) # Allocate memory

#--- Loop through chromosome names
j<-1 # chromosome counter
for (i in names(Hsapiens)[1:24]){


        # Data objects containing indices of S-V sites for each category are labeled with the chromosome name.
        # Here we copy the data objects to a set of variables with generic names which can be used consistently 
        # in the loop. 
        map<-get(paste0(i,'Map')) # Get the site map data for the current chrom
        ict<-    map[[2]]
        icl<-    map[[3]]
        iot<-    map[[4]]
        iol<-    map[[5]]
        insites<-map[[1]]

################################################
        # count in TSG exons for current chromosome (columns 1-4)
        ann_i <- exann[exann$chrom == i & exann$istsg==1,]
    
        sen        <-inrange(insites[ict[!is.na(ict[,1]),1],1],ann_i$start,ann_i$end) # Check if any Closed-Tight category sites are within the start-end range of ann_i
        antisen    <-inrange(insites[ict[!is.na(ict[,2]),2],2],ann_i$start,ann_i$end) 
        counts[j,1]<-length(which(sen)) + length(which(antisen))               # Fill an element of the counts table with the count
        
        sen        <-inrange(insites[icl[!is.na(icl[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[icl[!is.na(icl[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,2]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iot[!is.na(iot[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iot[!is.na(iot[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,3]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iol[!is.na(iol[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iol[!is.na(iol[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,4]<-length(which(sen)) + length(which(antisen))  

################################################
        # count in non-TSG exons for current chromosome (columns 5-8)
        ann_i <- exann[exann$chrom == i & exann$istsg==0,] # only difference from last block is setting exann$istsg==0 here (and different columns of 'counts')

        sen        <-inrange(insites[ict[!is.na(ict[,1]),1],1],ann_i$start,ann_i$end) # Check if any Closed-Tight category sites are within the start-end range of ann_i
        antisen    <-inrange(insites[ict[!is.na(ict[,2]),2],2],ann_i$start,ann_i$end) 
        counts[j,5]<-length(which(sen)) + length(which(antisen))                      # Fill an element of the counts table with the count
        
        sen        <-inrange(insites[icl[!is.na(icl[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[icl[!is.na(icl[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,6]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iot[!is.na(iot[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iot[!is.na(iot[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,7]<-length(which(sen)) + length(which(antisen))
        
        sen        <-inrange(insites[iol[!is.na(iol[,1]),1],1],ann_i$start,ann_i$end) 
        antisen    <-inrange(insites[iol[!is.na(iol[,2]),2],2],ann_i$start,ann_i$end)
        counts[j,8]<-length(which(sen)) + length(which(antisen))  
    
################################################
        # count in non-exons for current chromosome (columns 9-12)
        
        counts[j,9]   <- length(which(!is.na(as.vector(ict)))) - counts[j,1] - counts[j,5]
        counts[j,10]  <- length(which(!is.na(as.vector(icl)))) - counts[j,2] - counts[j,6]
        counts[j,11]  <- length(which(!is.na(as.vector(iot)))) - counts[j,3] - counts[j,7]
        counts[j,12]  <- length(which(!is.na(as.vector(iol)))) - counts[j,4] - counts[j,8]
    
        j<-j+1
        rm('sen','antisen')
}

In [37]:
counts

2713,5472,11420,39432,46985,105613,228957,812672,2622841,4089980,10248146,27118910
2352,5160,11039,34620,37507,91020,195883,665516,2747329,4668518,11573976,30198334
3177,6653,13964,46905,28745,67379,143562,494504,2281438,3924864,9737293,25445085
1219,3064,6358,21576,24456,58391,127386,405276,2213927,4008126,10072339,25611500
1232,2828,5849,20207,26986,62472,134077,446793,2062826,3615352,8968452,23484693
2077,4353,9234,29694,26223,63747,137065,464873,1974473,3396230,8463275,21844592
1406,3367,7100,24242,25117,57073,123541,424872,1890448,3029610,7589918,19654782
1416,3058,6415,21647,18621,43580,94079,314392,1624416,2802963,6985742,18325238
1236,2678,5710,19745,18054,43017,92306,330894,1393508,2252541,5579878,14791315
1188,2831,5717,19834,20928,50012,106943,365097,1509965,2414919,6007481,15971029
1637,3935,7993,30397,22374,51547,112689,412009,1436786,2383864,5973080,16142073


In [18]:
save(counts,file='../sim-develop/data')

[1]     2713     5472    11420    39432    46985   105613   228957   812672
 [9]  2622841  4089980 10248146 27118910

In [35]:
sum(counts[1,1:8])/sum(counts[1,])

[1] 0.02764565